<a href="https://colab.research.google.com/github/Forson12/CEM300-Natural-Language-Processing-Coursework/blob/main/2113122_CEM300_Natural_Language_Processing_Coursework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CEM300-Natural-Language-Processing-Coursework

In this work, I explore the Product Classification and Clustering Dataset.
<br>
Sources && Frameworks:

Dataset Chosen - https://archive.ics.uci.edu/dataset/837/product+classification+and+clustering

<br>


# ----------------------------------------------------
# Section 1 - Dataset
# ----------------------------------------------------

#Task 1: Loading the Dataset
# ------------------------------------------------------
 In this section, I will load and explore the Product Classification and Clustering Dataset taken from the UCI Machine Learning Repository.

I will be using pandas and scikit-learn to inspect its features.

My goal in this coursework is to use NLP techniques to represent the text and compare different algorithms for classifying products into their correct categories.

In [2]:

# Importing required libraries
import pandas as pd #using pandas to view data in dataframes
import io #io is used to load the data after uploading
from google.colab import files #we import the files package from google.colab framework to be able to upload files

# Uploading dataset (same idea as Lab 01)
uploaded = files.upload()  # expect 'pricerunner_aggregate.csv'

# Reading dataset into a dataframe
product_file = io.BytesIO(uploaded['pricerunner_aggregate.csv'])
product_df = pd.read_csv(product_file, header=None)

# Inspecting the dataframe
print("Shape of dataset:", product_df.shape)
product_df.head()


Saving pricerunner_aggregate.csv to pricerunner_aggregate.csv
Shape of dataset: (35312, 7)


,0,1,2,3,4,5,6
0,Product ID,Product Title,Merchant ID,Cluster ID,Cluster Label,Category ID,Category Label
1,1,apple iphone 8 plus 64gb silver,1,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
2,2,apple iphone 8 plus 64 gb spacegrau,2,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
3,3,apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim...,3,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
4,4,apple iphone 8 plus 64gb space grey,4,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones


## Task 1.1: Understanding the Dataset

This dataset, called the *Product Classification and Clustering Dataset*, contains records of retail products aggregated from online merchants.  
Each record includes a **Product ID**, textual information such as **Product Title** and **Cluster Label**, identifiers like **Merchant ID** and **Cluster ID**, and category information given by **Category ID** and **Category Label**.

The *Category Label* column serves as the target for classification, while the *Product Title* and *Cluster Label* provide textual data that require preprocessing before use in machine-learning models.  
Because these text fields are unstructured.

This section loads, inspects, and cleans the dataset.


In [ ]:
product_file = io.BytesIO(uploaded['pricerunner_aggregate.csv'])
product_df = pd.read_csv(product_file, header=None)
print(product_df)

                0                                                  1  \
0      Product ID                                      Product Title   
1               1                    apple iphone 8 plus 64gb silver   
2               2                apple iphone 8 plus 64 gb spacegrau   
3               3  apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim...   
4               4                apple iphone 8 plus 64gb space grey   
...           ...                                                ...   
35307       47350  smeg fab28 60cm retro style right hand hinge f...   
35308       47351  smeg fab28 60cm retro style left hand hinge fr...   
35309       47352  smeg fab28 60cm retro style left hand hinge fr...   
35310       47355     candy 60cm built under larder fridge cru160nek   
35311       47358           neff k4316x7gb built under larder fridge   

                  2            3                         4             5  \
0       Merchant ID   Cluster ID             Cluster Label 

The dataframe tells us a lot of useful information about the price runner dataset just uploaded.
Examples include:
1. There are 7 columns (6 features and 11 unique labels)
2. 35312 rows (as there is no header, this means there are 35312 examples in this dataset)
3. It has Columns including: Product ID, Merchant ID etc.

In [3]:
# --- Rename columns ---
product_df.columns = [
    "Product_ID", "Product_Title", "Merchant_ID",
    "Cluster_ID", "Cluster_Label", "Category_ID", "Category_Label"
]

# --- Check and handle missing values ---
print(product_df.isnull().sum())
product_df = product_df.dropna(subset=["Product_Title", "Category_Label"])

Product_ID        0
Product_Title     0
Merchant_ID       0
Cluster_ID        0
Cluster_Label     0
Category_ID       0
Category_Label    0
dtype: int64


## Task 1.3: Exploring Data Distribution

Before we perform representation learning in Section 2, we should understand how categories are distributed.  
This will confirm the dataset is balanced and ready for model training.


In [4]:
# --- Normalise text (preprocessing) ---
product_df["Product_Title"] = product_df["Product_Title"].astype(str).str.lower()
product_df["Cluster_Label"] = product_df["Cluster_Label"].astype(str).str.lower()

# --- Explore category distribution ---
print("Unique categories:", product_df["Category_Label"].nunique())
product_df["Category_Label"].value_counts().head(10)


Unique categories: 11


,count
Category_Label,
Fridge Freezers,5501
Mobile Phones,4081
Washing Machines,4044
CPUs,3862
Fridges,3584
TVs,3564
Dishwashers,3424
Digital Cameras,2697
Microwaves,2342
